### Mod 1 Project --- Data Movie

In [1]:
#top 50 lucrative movies from  tn_movie_budgets and bom_movie_gross   (find their ratings)
# most (top50) popular movies from tmdb_movies
# highest(top 50) ratings from imdb_title_ratings

# select common movies from above list
#find their genre, directors

### Import libraries & database

In [146]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

    100% |████████████████████████████████| 51kB 3.1MB/s ta 0:00:011
  Running setup.py bdist_wheel for python-Levenshtein ... done
  Stored in directory: /Users/yongweigao/Library/Caches/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [160]:
import pandas as pd
import numpy as np
import mysql.connector
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import string
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
%matplotlib inline

In [82]:
con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='movies')
cur = con.cursor()

# Data Cleaning

### IMDB Titles

In [83]:
query = ('''
SELECT *
FROM imdb_title_basics imdb_b
JOIN imdb_title_crew imdb_c USING (tconst)
JOIN imdb_title_ratings imdb_e USING (tconst)
''')
imdb_titles = pd.read_sql_query(query, con)
#joining all IMDB title tables using title ids and name ids

In [6]:
imdb_titles.isna().sum()
#checking for null values

tconst                 0
primary_title          0
original_title         0
start_year             0
runtime_minutes     7620
genres                 0
directors              0
writers            10561
averagerating          0
numvotes               0
dtype: int64

In [7]:
imdb_titles = imdb_titles.dropna(subset=['runtime_minutes'])
imdb_titles = imdb_titles.dropna(subset=['writers'])
#Removing rows with null values for runtime minutes and writers

In [8]:
pd.set_option('display.max_columns',None)
# Want to see all columns to see which are going to be useful

In [9]:
imdb_titles['genres'] = imdb_titles['genres'].map(lambda x: x.split(','))
imdb_titles['directors'] = imdb_titles['directors'].map(lambda x: x.split(','))
imdb_titles['writers'] = imdb_titles['writers'].map(lambda x: x.split(','))
# Converting genres, directors, writers to lists, so we can easily check most popular later.

In [10]:
imdb_titles.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"[Action, Crime, Drama]",[nm0712540],"[nm0023551, nm1194313, nm0347899, nm1391276]",7.0,77
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,[Drama],[nm0000080],"[nm0000080, nm0462648]",6.9,4517
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"[Comedy, Drama, Fantasy]","[nm0765384, nm0749914]","[nm1360635, nm0749914]",6.5,119
6,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"[Adventure, Animation, Comedy]",[nm0365480],[nm0365480],8.1,263
7,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,[Drama],[nm1030585],"[nm1030585, nm1921430]",6.8,451


In [11]:
imdb_titles['age_of_movie']= 2020 - imdb_titles['start_year']
#Creating new column to have "age of movie" as a reference

### IMDB Crew

In [12]:
query = ('''
SELECT *
FROM imdb_name_basics
''')
imdb_crew = pd.read_sql_query(query, con)

In [13]:
imdb_crew.isna().sum()
# Checking for null values

nconst                     0
primary_name               0
birth_year            523912
death_year            599865
primary_profession         0
known_for_titles           0
dtype: int64

In [14]:
imdb_crew = imdb_crew.fillna(0)
#Replacing null values with 0

In [15]:
imdb_crew['birth_year'] = imdb_crew['birth_year'].map(lambda x: int(x))
imdb_crew['death_year'] = imdb_crew['death_year'].map(lambda x: int(x))
#Converting birth years and death years to integer

In [16]:
imdb_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
nconst                606648 non-null object
primary_name          606648 non-null object
birth_year            606648 non-null int64
death_year            606648 non-null int64
primary_profession    606648 non-null object
known_for_titles      606648 non-null object
dtypes: int64(2), object(4)
memory usage: 27.8+ MB


In [17]:
imdb_crew['primary_profession'] = imdb_crew['primary_profession'].map(lambda x: x.split(','))
imdb_crew['known_for_titles'] = imdb_crew['known_for_titles'].map(lambda x: x.split(','))
#Converting these to lists so we can easily reference them later

In [18]:
imdb_crew.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0038355, tt0117057, tt0071877, tt0037382]"
1,nm0000003,Brigitte Bardot,1934,0,"[actress, soundtrack, producer]","[tt0049189, tt0057345, tt0054452, tt0059956]"
2,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0069467, tt0050986, tt0083922, tt0050976]"
3,nm0000006,Ingrid Bergman,1915,1982,"[actress, soundtrack, producer]","[tt0071877, tt0036855, tt0038109, tt0038787]"
4,nm0000007,Humphrey Bogart,1899,1957,"[actor, soundtrack, producer]","[tt0034583, tt0033870, tt0037382, tt0043265]"


In [19]:
imdb_crew['age'] = imdb_crew.apply(lambda row: (2020 - row['birth_year']) if row['death_year'] == 0 else None, axis=1)
#Adding an age column. If the age is 2020, it means that there is no birth year available. If the age is equal to None, it means the person is dead.

In [20]:
imdb_crew.head(100)

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,age
0,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0038355, tt0117057, tt0071877, tt0037382]",NaN
1,nm0000003,Brigitte Bardot,1934,0,"[actress, soundtrack, producer]","[tt0049189, tt0057345, tt0054452, tt0059956]",86.0
2,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0069467, tt0050986, tt0083922, tt0050976]",NaN
3,nm0000006,Ingrid Bergman,1915,1982,"[actress, soundtrack, producer]","[tt0071877, tt0036855, tt0038109, tt0038787]",NaN
4,nm0000007,Humphrey Bogart,1899,1957,"[actor, soundtrack, producer]","[tt0034583, tt0033870, tt0037382, tt0043265]",NaN
5,nm0000008,Marlon Brando,1924,2004,"[actor, soundtrack, director]","[tt0078788, tt0070849, tt0047296, tt0068646]",NaN
6,nm0000010,James Cagney,1899,1986,"[actor, soundtrack, director]","[tt0029870, tt0055256, tt0035575, tt0031867]",NaN
7,nm0000015,James Dean,1931,1955,"[actor, miscellaneous]","[tt0049261, tt0045458, tt0048028, tt0048545]",NaN
8,nm0000016,Georges Delerue,1925,1992,"[composer, soundtrack, music_department]","[tt0091763, tt0080610, tt0069946, tt0096320]",NaN
9,nm0000018,Kirk Douglas,1916,0,"[actor, producer, soundtrack]","[tt0049456, tt0080736, tt0054331, tt0052365]",104.0


### BOM

In [21]:
query = ('''
SELECT * FROM bom_movie_gross
''')
bom = pd.read_sql_query(query, con)

In [22]:
#drop Null value in domestic_gross
bom = bom.dropna(subset =['domestic_gross'])

#convert Null value in foreign_gross to 0
bom.foreign_gross = bom.foreign_gross.fillna(0)

#convert domestic_gross and foreign_gross from float to int
bom.domestic_gross = bom.domestic_gross.map(lambda x: int(x))   
bom.foreign_gross = bom.foreign_gross.map(lambda x: int(x))  

In [23]:
#remove spaces on both end of strings
bom.title = bom.title.map(lambda x: x.strip())
bom.studio = bom.studio.map(lambda x: x.strip())

In [189]:
#convert year to str
bom['year'] = bom['year'].map(str)

#remove all punctuation and numbers inside movie title
translator1 = str.maketrans('0123456789','..........')
translator2 = str.maketrans('', '', string.punctuation)
bom['title_cleaned'] = bom.title.map(lambda x: x.translate(translator1).translate(translator2).strip())

#combine title and year
bom['title_cleaned'] = bom['title_cleaned'] + '_' + bom['year']

#replace all empty spaces with _ and lower all string
bom['title_cleaned'] = bom['title_cleaned'].str.replace(' ','_').str.lower()

#convert year back to int
bom['year'] = bom['year'].map(int)

In [190]:
bom.head()

,title,studio,domestic_gross,foreign_gross,year,title_cleaned
0,Toy Story 3,BV,415000000,652000000,2010,toy_story_2010
1,Alice in Wonderland (2010),BV,334200000,691300000,2010,alice_in_wonderland_2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000,664300000,2010,harry_potter_and_the_deathly_hallows_part_2010
3,Inception,WB,292600000,535700000,2010,inception_2010
4,Shrek Forever After,P/DW,238700000,513900000,2010,shrek_forever_after_2010


### TN

In [25]:
query_tn = ('''
SELECT * FROM tn_movie_budgets
''')
tn = pd.read_sql_query(query_tn, con)

In [26]:
# convert release_date to date format
tn.release_date = tn.release_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y'))

# convert production_budget, domestic_gross, worldwide_gross to date format
tn.production_budget = tn.production_budget.map(lambda x: int(x.replace(',','').replace('$','')))
tn.domestic_gross = tn.domestic_gross.map(lambda x: int(x.replace(',','').replace('$','')))
tn.worldwide_gross = tn.worldwide_gross.map(lambda x: int(x.replace(',','').replace('$','')))

#add profit column
tn['profit'] = tn.worldwide_gross - tn.production_budget

In [187]:
#add new column for release year
tn['year'] = tn.release_date.map(lambda x: str(x.year))

#remove all punctuation and numbers inside movie title
translator1 = str.maketrans('0123456789','..........')
translator2 = str.maketrans('', '', string.punctuation)
tn['title_cleaned'] = tn.movie.map(lambda x: x.translate(translator1).translate(translator2).strip())

#combine title and year
tn['title_cleaned'] = tn['title_cleaned']+'_'+tn['year']

#replace all empty spaces with _ and lower all string
tn['title_cleaned'] = tn['title_cleaned'].str.replace(' ','_').str.lower()

#drop column year
tn = tn.drop(axis = 1, columns='year')

In [188]:
tn.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit,title_cleaned
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2351345279,avatar_2009
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,pirates_of_the_caribbean_on_stranger_tides_2011
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,-200237650,dark_phoenix_2019
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,avengers_age_of_ultron_2015
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,star_wars_ep_viii_the_last_jedi_2017


### TMDB

In [29]:
query_tmdb = ('''
SELECT * FROM tmdb_movies
''')
tmdb = pd.read_sql_query(query_tmdb, con)

In [30]:
query_tmdb_link = ('''
SELECT * FROM tmdb_imdb_link
''')
tmdb_link = pd.read_sql_query(query_tmdb_link, con)

In [31]:
#convert genre_ids to int
tmdb.genre_ids.map(lambda x: int(x))

#remove spaces on both end of strings
tmdb.original_language = tmdb.original_language.map(lambda x: x.strip())
tmdb.original_title = tmdb.original_title.map(lambda x: x.strip())
tmdb.title = tmdb.title.map(lambda x: x.strip())

In [32]:
#Join tmdb and tmdb_link tables
tmdb = tmdb.join(tmdb_link.set_index('tmdb_id'), on = 'id', how = 'left')

In [33]:
#clean the joined table
tmdb.imdb_id = tmdb.imdb_id.fillna(0).map(lambda x: int(x))

In [192]:
#add new column for release year
tmdb['year'] = tmdb.release_date.map(lambda x: str(x.year))

#remove all punctuation and numbers inside movie title
translator1 = str.maketrans('0123456789','..........')
translator2 = str.maketrans('', '', string.punctuation)
tmdb['title_cleaned'] = tmdb.title.map(lambda x: x.translate(translator1).translate(translator2).strip())

#combine title and year
tmdb['title_cleaned'] = tmdb['title_cleaned']+'_'+tmdb['year']

#replace all empty spaces with _ and lower all string
tmdb['title_cleaned'] = tmdb['title_cleaned'].str.replace(' ','_').str.lower()

#drop column year
tmdb = tmdb.drop(axis = 1, columns='year')

In [193]:
tmdb.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,imdb_id,title_cleaned
0,0,12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,926084,harry_potter_and_the_deathly_hallows_part_2010
1,1,10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,892769,how_to_train_your_dragon_2010
2,2,10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,1228705,iron_man_2010
3,3,862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,114709,toy_story_1995
4,4,27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,1375666,inception_2010


### RT

In [35]:
query_rt_info = ('''
SELECT * FROM rt_movie_info
''')
rt_info = pd.read_sql_query(query_rt_info, con)

In [36]:
query_rt_reviews = ('''
SELECT * FROM rt_reviews
''')
rt_reviews = pd.read_sql_query(query_rt_reviews, con)

In [37]:
query_rt_link = ('''
SELECT * FROM rt_imdb_link
''')
rt_link = pd.read_sql_query(query_rt_link, con)

In [38]:
rt_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
id              1560 non-null int64
synopsis        1560 non-null object
rating          1560 non-null object
genre           1560 non-null object
director        1560 non-null object
writer          1560 non-null object
theater_date    1560 non-null object
dvd_date        1560 non-null object
currency        1560 non-null object
box_office      1560 non-null object
runtime         1560 non-null object
studio          494 non-null object
dtypes: int64(1), object(11)
memory usage: 146.3+ KB


### RT_Info Cleaning

In [39]:
#replace empty value in theater_date and dvd_date with Jan 1,1000
rt_info['theater_date'].loc[rt_info.theater_date == ''] = 'Jan 1, 1000'
rt_info['dvd_date'].loc[rt_info.dvd_date == ''] = 'Jan 1, 1000'

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [40]:
#convert theater_date and dvd_date to datetime format
rt_info.theater_date = rt_info.theater_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y').date())
rt_info.dvd_date = rt_info.dvd_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y').date())

In [41]:
#clean box_office date
rt_info['box_office'].loc[rt_info.box_office == ''] = '0'
rt_info['box_office'] = rt_info.box_office.map(lambda x: int(x.replace(',', '')))

In [42]:
#clean runtime (replace empty value with 0)
rt_info['runtime'].loc[rt_info.runtime == ''] = '0'
rt_info['runtime'] = rt_info.runtime.map(lambda x: int(x.replace('minutes', '')))

In [43]:
#remove spaces on both end of strings
rt_info.synopsis = rt_info.synopsis.map(lambda x: x.strip())
rt_info.rating = rt_info.rating.map(lambda x: x.strip())
rt_info.genre = rt_info.genre.map(lambda x: x.strip())
rt_info.director = rt_info.director.map(lambda x: x.strip())
rt_info.writer = rt_info.writer.map(lambda x: x.strip())

In [44]:
rt_info['genre'] = rt_info['genre'].map(lambda x: x.split('|'))
rt_info['writer'] = rt_info['writer'].map(lambda x: x.split('|'))
rt_info['director'] = rt_info['director'].map(lambda x: x.split('|'))
#Converting genre, writer, director to list so we can reference them easily

In [45]:
rt_info = rt_info.drop(columns=['box_office','currency'],axis=1)
#Dropping box_office because we have better data on another table, dropping currency because we removed box office data

### RT_Reviews Cleaning

In [46]:
#convert date to datetime format
rt_reviews.date = rt_reviews.date.map(lambda x: dt.datetime.strptime(x, '%B %d, %Y').date())

In [47]:
#convert top_critic to int
rt_reviews.top_critic = rt_reviews.top_critic.map(lambda x: int(x))

In [48]:
#remove spaces on both end of strings
rt_reviews.review = rt_reviews.review.map(lambda x: x.strip())
rt_reviews.rating = rt_reviews.rating.map(lambda x: x.strip())
rt_reviews.fresh = rt_reviews.fresh.map(lambda x: x.strip())
rt_reviews.critic = rt_reviews.critic.map(lambda x: x.strip())
rt_reviews.publisher = rt_reviews.publisher.map(lambda x: x.strip())

In [49]:
rt_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12300 entries, 0 to 12299
Data columns (total 8 columns):
id            12300 non-null int64
review        12300 non-null object
rating        12300 non-null object
fresh         12300 non-null object
critic        12300 non-null object
top_critic    12300 non-null int64
publisher     12300 non-null object
date          12300 non-null object
dtypes: int64(2), object(6)
memory usage: 768.8+ KB


### RT_link cleaning

In [50]:
rt_link.year = rt_link.year.fillna(1000)

In [51]:
rt_link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17468 entries, 0 to 17467
Data columns (total 6 columns):
imdbid         17468 non-null int64
rtid           17468 non-null int64
title          17468 non-null object
year           17468 non-null float64
mpaa_rating    17468 non-null object
runtime        17468 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 818.9+ KB


In [52]:
#replace empty space to year 1000 in year
rt_link.year = rt_link.year.fillna(1000)

#convert date to datetime format
rt_link.year = rt_link.year.map(lambda x: (int(x)))

In [53]:
#replace empty space in runtime to 0
rt_link['runtime'].loc[rt_link.runtime == ''] = '0'

#convert runtime to int format
rt_link.runtime = rt_link.runtime.map(lambda x: (int(x)))

In [199]:
#convert year to str
rt_link['year'] = rt_link['year'].map(str)

#remove all punctuation and numbers inside movie title
translator1 = str.maketrans('0123456789','..........')
translator2 = str.maketrans('', '', string.punctuation)
rt_link['title_cleaned'] = rt_link.title.map(lambda x: x.translate(translator1).translate(translator2).strip())

#combine title and year
rt_link['title_cleaned'] = rt_link['title_cleaned'] + '_' + rt_link['year']

#replace all empty spaces with _ and lower all string
rt_link['title_cleaned'] = rt_link['title_cleaned'].str.replace(' ','_').str.lower()

#convert year back to int
rt_link['year'] = rt_link['year'].map(int)

In [202]:
rt_link.head()

,imdbid,rtid,title,year,mpaa_rating,runtime,title_cleaned
0,4,497213641,Un bon bock (A Good Beer),1892,Unrated,0,un_bon_bock_a_good_beer_1892
1,417,376896599,Le Voyage dans la lune (A Trip to the Moon),1999,Unrated,17,le_voyage_dans_la_lune_a_trip_to_the_moon_1999
2,516,771243433,El hotel electrico (The Electric Hotel),1908,Unrated,8,el_hotel_electrico_the_electric_hotel_1908
3,2445,771233754,Quo Vadis?,1913,Unrated,45,quo_vadis_1913
4,2461,770678720,Richard III (The Life and Death of King Richar...,1912,Unrated,55,richard_iii_the_life_and_death_of_king_richard...


# Top 200 Profitable Movies after 2000


### Both BOM and TN have similar information about budget, but TN database is more complete so we decided to use TN to calculate profit.

In [71]:
# get top 200 revenue domestic movies after 2000
top200_revenue_domestic = tn.loc[tn.release_date>dt.datetime(2000,1,1)].sort_values('domestic_gross',ascending=False).head(200)

# get top 200 revenue worldwide movies after 2000
top200_revenue_world = tn.loc[tn.release_date>dt.datetime(2000,1,1)].sort_values('worldwide_gross',ascending=False).head(200)

# get top 200 profit movies after 2000
top200_profit = tn.loc[tn.release_date>dt.datetime(2000,1,1)].sort_values('profit',ascending=False).head(200)

In [131]:
top200_profit.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2351345279
6,7,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,1748134200
5,6,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,1747311220
33,34,2015-06-12,Jurassic World,215000000,652270625,1648854864,1433854864
66,67,2015-04-03,Furious 7,190000000,353007020,1518722794,1328722794


In [132]:
# there are only 140 movies from top200_profit that can be find inside imdb datebdase. 
# So we narrowed our data to these 140 movies

top140_profit_imdb = imdb_titles.loc[imdb_titles.primary_title.isin(list(top200_profit.movie))]

In [153]:
fuzz.ratio('ss','ss')

100

In [155]:
count=0
for i in top200_profit.movie:
    for j in imdb_titles.primary_title:
        if fuzz.ratio(i,j)>80:
            print(i,j)
            count +=1


Avatar Avatar
Avengers: Infinity War Avengers: Infinity War
Star Wars Ep. VII: The Force Awakens Star Wars: Episode VII - The Force Awakens
Jurassic World Jurassic World
Furious 7 Furious
Furious 7 Furious 7
Furious 7 Furious
Furious 7 Furious
The Avengers The Avengers
The Avengers The Adventurers
The Avengers The Avenue
Harry Potter and the Deathly Hallows: Part II Harry Potter and the Deathly Hallows: Part 1
Harry Potter and the Deathly Hallows: Part II Harry Potter and the Deathly Hallows: Part 2
Black Panther The Black Panther
Black Panther Black Panther
Black Panther Black Paper
Black Panther Black Panther Woman
Black Panther Black Water
Jurassic World: Fallen Kingdom Jurassic World: Fallen Kingdom
Frozen Frozen
Frozen Frozen
Frozen Frozen
Beauty and the Beast Beauty and the Breast
Beauty and the Beast Beauty and the Beast
Beauty and the Beast Beauty and the Beast
Beauty and the Beast Beauty and the Bestie
Beauty and the Beast Beauty and the Dogs
Minions Minions
Avengers: Age of U

KeyboardInterrupt: 

In [134]:
# drop original title
top140_profit_imdb = top140_profit_imdb.drop(axis = 1, columns = 'original_title')

In [135]:
top140_profit_imdb.head()

,tconst,primary_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes
52,tt0369610,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",nm1119880,"nm0415425,nm0798646,nm1119880,nm2081046,nm0000341",7.0,539338
91,tt0435761,Toy Story 3,2010,103.0,"Adventure,Animation,Comedy",nm0881279,"nm0005124,nm0004056,nm0881279,nm1578335",8.3,682218
109,tt0448694,Puss in Boots,2011,90.0,"Action,Adventure,Animation",nm3735491,"nm1117226,nm0528244,nm0204030,nm0011470,nm0674...",6.6,133355
114,tt0451279,Wonder Woman,2017,141.0,"Action,Adventure,Fantasy",nm0420941,"nm0374302,nm0811583,nm0297229,nm0551376,nm4243936",7.5,487527
119,tt0454876,Life of Pi,2012,127.0,"Adventure,Drama,Fantasy",nm0000487,"nm1674631,nm1341735",7.9,535836


In [140]:
#join tn and imdb database together for top 140 movies
top140_profit = top140_profit_imdb.merge(
    top200_profit, left_on='primary_title', right_on ='movie').drop(
    axis=1, columns=['start_year', 'id'])

In [142]:
top140_profit.head()

,tconst,primary_title,runtime_minutes,genres,directors,writers,averagerating,numvotes,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit
0,tt0369610,Jurassic World,124.0,"Action,Adventure,Sci-Fi",nm1119880,"nm0415425,nm0798646,nm1119880,nm2081046,nm0000341",7.0,539338,2015-06-12,Jurassic World,215000000,652270625,1648854864,1433854864
1,tt0435761,Toy Story 3,103.0,"Adventure,Animation,Comedy",nm0881279,"nm0005124,nm0004056,nm0881279,nm1578335",8.3,682218,2010-06-18,Toy Story 3,200000000,415004880,1068879522,868879522
2,tt0448694,Puss in Boots,90.0,"Action,Adventure,Animation",nm3735491,"nm1117226,nm0528244,nm0204030,nm0011470,nm0674...",6.6,133355,2011-10-28,Puss in Boots,130000000,149260504,554987477,424987477
3,tt0451279,Wonder Woman,141.0,"Action,Adventure,Fantasy",nm0420941,"nm0374302,nm0811583,nm0297229,nm0551376,nm4243936",7.5,487527,2017-06-02,Wonder Woman,150000000,412563408,821133378,671133378
4,tt4028068,Wonder Woman,60.0,Sci-Fi,"nm3645818,nm4197194","nm3645818,nm5566809,nm4197194",4.2,20,2017-06-02,Wonder Woman,150000000,412563408,821133378,671133378


### 

# Top 200 Highest Rated Movies after 2000